In [1]:
import py2neo
import pandas as pd
from backend_utils import *

In [2]:
data_loc = "./data/"
gwea = GWEA(data=EAData(loc=data_loc))

In [3]:
len(gwea.data.kg[0].rels)

70414

In [10]:
# 获取所有节点
'''
tx = graph.begin()
nodes = {}
for i in range(10):
    node = py2neo.Node("ZH", id = i, name = f"Test{str(i)}")
    nodes[f"Test{str(i)}"] = node
'''
nodes = {}
for rel in gwea.data.kg[0].rels:
    if rel[0] not in nodes.keys():
        nodes[rel[0]] = py2neo.Node('ZH', id = rel[0], name = gwea.data.kg[0].ent_ids[rel[0]])
    if rel[2] not in nodes.keys():
        nodes[rel[2]] = py2neo.Node('ZH', id = rel[2], name = gwea.data.kg[0].ent_ids[rel[2]])
for rel in gwea.data.kg[1].rels:
    if rel[0] not in nodes.keys():
        nodes[rel[0]] = py2neo.Node('EN', id = rel[0], name = gwea.data.kg[1].ent_ids[rel[0]])
    if rel[2] not in nodes.keys():
        nodes[rel[2]] = py2neo.Node('EN', id = rel[2], name = gwea.data.kg[1].ent_ids[rel[2]])
len(nodes)


38960

In [16]:
# 获取所有边
''' 
rels = []
for i in range(9):
    if f"Test{str(i)}" in nodes.keys() and f"Test{str(i+1)}" in nodes.keys():
        rel = py2neo.Relationship(nodes[f"Test{str(i)}"], "TestRel", nodes[f"Test{str(i+1)}"])
        rels.append(rel)
'''
rels = []
for rel in gwea.data.kg[0].rels:
    if rel[0] in nodes.keys() and rel[2] in nodes.keys():
        new_rel = py2neo.Relationship(nodes[rel[0]], gwea.data.kg[0].rel_ids[rel[1]], nodes[rel[2]])
        rels.append(new_rel)
for rel in gwea.data.kg[1].rels:
    if rel[0] in nodes.keys() and rel[2] in nodes.keys():
        new_rel = py2neo.Relationship(nodes[rel[0]], gwea.data.kg[1].rel_ids[rel[1]], nodes[rel[2]])
        rels.append(new_rel)
len(rels)

165556

In [18]:
len(rels)

165556

In [19]:
url = "http://127.0.0.1:7474"
username = "neo4j"
password = "kgdemo"
graph = py2neo.Graph(url, auth=(username, password))

In [20]:
def clear_database():
    cypher = "match (n) detach delete n"
    graph.run(cypher)

In [21]:
clear_database()

In [ ]:
tx = graph.begin()
sub = py2neo.Subgraph(list(nodes.values()),rels)
tx.create(sub)
graph.commit(tx)

In [25]:
cypher = "match (n) return n limit 25"
df = graph.run(cypher).to_data_frame()

In [34]:
df['n'].iloc[0]['name']

'Zhou Qiang'

In [52]:
def get_force_graph_data(graph, id1, id2):
    def get_subgraph(graph, id):
            cypher = f'''
                MATCH (n) where n.id={id} \
                CALL apoc.path.subgraphAll(n, {{\
                    maxLevel: 1\
                }}) \
                YIELD relationships \
                unwind relationships as r_ WITH DISTINCT r_ \
                return type(r_) as r_name, id(r_) as r_id, \
                startNode(r_).id as x_id, startNode(r_).name as x_name, \
                endNode(r_).id as y_id, endNode(r_).name as y_name, \
                labels(startNode(r_)) as x_labels, labels(endNode(r_)) as y_labels
            '''
            df = graph.run(cypher).to_data_frame()
            rel_list = df.values.tolist()
            nodes = {}
            edges = {}
            for rel in rel_list:
                if rel[2] not in nodes.keys():
                    nodes[rel[2]] = rel[3]
                if rel[4] not in nodes.keys():
                    nodes[rel[4]] = rel[5]
                if tuple([rel[2], rel[4]]) not in edges.keys():
                    edges[tuple([rel[2], rel[4]])] = [rel[0]]
                else:
                    edges[tuple([rel[2], rel[4]])].append(rel[0])
            res = {
                "nodes":[],
                "edges":[]
            }
            for key, value in nodes.items():
                res["nodes"].append({
                    "id" : key,
                    "name": value
                })
            for key, value in edges.items():
                res["edges"].append({
                    "source" : key[0],
                    "target": key[1],
                    "rels" : value
                })
            return res
    return [get_subgraph(graph,id1),get_subgraph(graph, id2)]
get_force_graph_data(graph, 0, 10500)

{'nodes': [{'id': 10500, 'name': 'Abdullah Gül'},
  {'id': 13972, 'name': 'University of Exeter'},
  {'id': 10903, 'name': 'Bülent Ecevit'},
  {'id': 16941, 'name': 'Istanbul University'},
  {'id': 20308, 'name': 'Turkey'},
  {'id': 19238, 'name': 'Recep Tayyip Erdoğan'},
  {'id': 12009, 'name': 'Grand National Assembly of Turkey'}],
 'edges': [{'source': 10500, 'target': 13972, 'rels': ['almaMater']},
  {'source': 10500, 'target': 10903, 'rels': ['predecessor']},
  {'source': 10500, 'target': 16941, 'rels': ['almaMater']},
  {'source': 10500, 'target': 20308, 'rels': ['birthPlace']},
  {'source': 10500, 'target': 19238, 'rels': ['successor', 'primeminister']},
  {'source': 10500, 'target': 12009, 'rels': ['office']},
  {'source': 10903, 'target': 20308, 'rels': ['birthPlace', 'deathPlace']},
  {'source': 10903, 'target': 10500, 'rels': ['after', 'successor']},
  {'source': 10903, 'target': 12009, 'rels': ['office']},
  {'source': 16941, 'target': 20308, 'rels': ['country']},
  {'sourc

In [43]:
rel_list = df.values.tolist()
rel_list[0]

['almaMater',
 109193,
 10500,
 'Abdullah Gül',
 13972,
 'University of Exeter',
 ['EN'],
 ['EN']]

In [44]:
nodes = {}
edges = {}
for rel in rel_list:
    if rel[2] not in nodes.keys():
        nodes[rel[2]] = rel[3]
    if rel[4] not in nodes.keys():
        nodes[rel[4]] = rel[5]
    if tuple([rel[2], rel[4]]) not in edges.keys():
        edges[tuple([rel[2], rel[4]])] = [rel[0]]
    else:
        edges[tuple([rel[2], rel[4]])].append(rel[0])

In [47]:
res = {
    "nodes":[],
    "edges":[]
}
for key, value in nodes.items():
    res["nodes"].append({
        "id" : key,
        "name": value
    })
for key, value in edges.items():
    res["edges"].append({
        "source" : key[0],
        "target": key[1],
        "rels" : value
    })

In [48]:
res

{'nodes': [{'id': 10500, 'name': 'Abdullah Gül'},
  {'id': 13972, 'name': 'University of Exeter'},
  {'id': 10903, 'name': 'Bülent Ecevit'},
  {'id': 16941, 'name': 'Istanbul University'},
  {'id': 20308, 'name': 'Turkey'},
  {'id': 19238, 'name': 'Recep Tayyip Erdoğan'},
  {'id': 12009, 'name': 'Grand National Assembly of Turkey'}],
 'edges': [{'source': 10500, 'target': 13972, 'rels': ['almaMater']},
  {'source': 10500, 'target': 10903, 'rels': ['predecessor']},
  {'source': 10500, 'target': 16941, 'rels': ['almaMater']},
  {'source': 10500, 'target': 20308, 'rels': ['birthPlace']},
  {'source': 10500, 'target': 19238, 'rels': ['successor', 'primeminister']},
  {'source': 10500, 'target': 12009, 'rels': ['office']},
  {'source': 10903, 'target': 20308, 'rels': ['birthPlace', 'deathPlace']},
  {'source': 10903, 'target': 10500, 'rels': ['after', 'successor']},
  {'source': 10903, 'target': 12009, 'rels': ['office']},
  {'source': 16941, 'target': 20308, 'rels': ['country']},
  {'sourc